Since there is a limit of 1000 requests per hour, we will use a SQLite database to store the results of the geocoding.
And we need to run this script until there are no null values in the lat and lng columns.

In [15]:
import sqlite3
import pandas as pd

In [16]:
db_file_name = "db/sample-superstore.sqlite"
con = sqlite3.connect(db_file_name)
cur = con.cursor()

In [17]:
def q(sql, cur, convert_to_df=True):
    cur.execute(sql)
    r = cur.fetchall()
    if convert_to_df:
        r.insert(0, next(zip(*cur.description)))
        #display(tabulate.tabulate(r, tablefmt="html", headers="firstrow"))
        return pd.DataFrame(r[1:], columns=r[0])
    else:
        return r

In [18]:
sql = """SELECT COUNT(*) FROM cities"""
q(sql, cur)

,COUNT(*)
0,3636


In [19]:
sql = """SELECT
    cities.name AS city_name,
    countries.name AS country_name,
    cities.id AS city_id,
    cities.lat AS lat,
    cities.lng AS lng
    FROM cities
    INNER JOIN countries ON cities.country_id = countries.id
 WHERE lng IS NULL AND lat IS NULL
 LIMIT 1000"""
r = q(sql, cur, convert_to_df=False)
display(r)

[('Abidjan', "Cote d'Ivoire", 12, None, None),
 ('Bouake', "Cote d'Ivoire", 465, None, None),
 ('Daloa', "Cote d'Ivoire", 861, None, None),
 ('Gagnoa', "Cote d'Ivoire", 1137, None, None),
 ('Korhogo', "Cote d'Ivoire", 1695, None, None),
 ('Man', "Cote d'Ivoire", 1987, None, None),
 ('Yamoussoukro', "Cote d'Ivoire", 3557, None, None)]

In [20]:
from geopy import geocoders
gn = geocoders.GeoNames(username='temnovit')

In [21]:
i = 1
for row in r:
    city_country = row[0] + ", " + row[1]
    geo = gn.geocode(city_country)
    if geo:
        print(f"{i}. {city_country} -> {geo.latitude}, {geo.longitude}")
        sql = """UPDATE cities SET lat = ?, lng = ? WHERE id = ?"""
        cur.execute(
            sql,
            (
                geo.latitude,
                geo.longitude,
                row[2]
            )
        )
    else:
        print(f"{i}. {city_country} -> NO DATA")
    i += 1
con.commit()

1. Abidjan, Cote d'Ivoire -> NO DATA
2. Bouake, Cote d'Ivoire -> NO DATA
3. Daloa, Cote d'Ivoire -> NO DATA
4. Gagnoa, Cote d'Ivoire -> NO DATA
5. Korhogo, Cote d'Ivoire -> NO DATA
6. Man, Cote d'Ivoire -> NO DATA
7. Yamoussoukro, Cote d'Ivoire -> NO DATA
